## Guidelines:

To be able to run this Jupyter notebook requires an installed R distribution and the relevant packages. This is because the Python libraries pymer4 and rpy2 interface with R behind the scenes.

* The lme4 package is required for the lmer() function.
* The nlme package is required for the lme() function.

However the nmle functions are currently depreciated, and cannot be run at the same time as the lme4 functions.  (The error "*namespace 'nlme' is imported by 'lme4' so cannot be unloaded*" occurs).  
Therefore each package must firstly be removed from the R environment in order to install and access the other, e.g. using commands in R:  

    remove.packages("lme4") 
    install.packages("nlme")
    
Alternatively, the Jupyter notebook output can simply be referenced, without needing to run the code cells.

## Importing the dataset

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import seaborn as sns
import scipy as sp
from scipy import stats
import statsmodels.api as sm
import statsmodels.formula.api as smf
from pymer4.models import Lmer
import rpy2

Unable to determine R home: [WinError 2] The system cannot find the file specified


In [3]:
data = pd.read_csv("../Main Study (Face)/main_study_face_data_modified.csv")
data["participant_id"] = data["participant_id"].astype("category")

Renaming the columns for ease of reading the model results:

In [4]:
data.columns=["participant_id", "prime condition", "prime_", "trial", "image", "time", "chose_masc"]

Ensuring that the neutral condition will be used as the (first) reference condition in the model.

In [5]:
data["prime_"] = data["prime_"].replace({"neutral":"_neutral"})

Importing the dataset without the duplicated participants' data:

In [6]:
b_participants = [p_id for p_id in list(data["participant_id"].unique()) if "b" in p_id and p_id[:-1] in list(data["participant_id"].unique())]
data_no_b = data[~data["participant_id"].isin(b_participants)]
data_no_b = data_no_b.reset_index(drop=True)

Without the first 5 trials:

In [7]:
no_5_trial = data[data["trial"].isin(range(6, 41))]

In [8]:
data

,participant_id,prime condition,prime_,trial,image,time,chose_masc
0,11,1,_neutral,2,Slide11.bmp,0,0
1,11,1,_neutral,3,Slide9.bmp,0,1
2,11,1,_neutral,4,Slide5.bmp,0,0
3,11,1,_neutral,5,Slide1.bmp,0,1
4,11,1,_neutral,6,Slide18.bmp,0,1
...,...,...,...,...,...,...,...
12659,75b,5,pathogen,36,Slide20.bmp,1,1
12660,75b,5,pathogen,37,Slide2.bmp,1,1
12661,75b,5,pathogen,38,Slide10.bmp,1,1
12662,75b,5,pathogen,39,Slide14.bmp,1,0


## Study's main model (random intercept model)

Fitting the main model described in the study paper, to each of the datasets:

### Using lme

In [28]:
%load_ext rpy2.ipython

The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython


These model and results match that in the original paper exactly:

In [ ]:
# original data

In [45]:
%%R -i data
library(nlme)
model <- lme(chose_masc ~ time*prime_, random=~1|participant_id, data=data)
summary(model)

Linear mixed-effects model fit by REML
  Data: data 
       AIC   BIC    logLik
  17676.66 17766 -8826.328

Random effects:
 Formula: ~1 | participant_id
        (Intercept)  Residual
StdDev:    0.140347 0.4756268

Fixed effects:  chose_masc ~ time * prime_ 
                            Value  Std.Error    DF   t-value p-value
(Intercept)             0.5571658 0.02160315 12328 25.790946  0.0000
time                    0.0650744 0.01862048 12328  3.494773  0.0005
prime_male group       -0.0445560 0.03104092   326 -1.435395  0.1521
prime_male/female      -0.0114496 0.03054416   326 -0.374853  0.7080
prime_male/male        -0.0772700 0.03069174   326 -2.517614  0.0123
prime_pathogen         -0.0126368 0.03102619   326 -0.407295  0.6841
time:prime_male group   0.0178697 0.02678034 12328  0.667269  0.5046
time:prime_male/female -0.0650238 0.02635278 12328 -2.467437  0.0136
time:prime_male/male    0.0040213 0.02649672 12328  0.151765  0.8794
time:prime_pathogen    -0.0208333 0.02676700 12328 

In [10]:
# anova on original data
%%R -i data
library(nlme)
model <- lme(chose_masc ~ time*prime_, random=~1|participant_id, data=data)
anova(model, type="marginal", adjustSigma=F)

            numDF denDF  F-value p-value
(Intercept)     1 12328 665.1729  <.0001
time            1 12328  12.2134  0.0005
prime_          4   326   2.0980  0.0808
time:prime_     4 12328   2.9559  0.0188


In [ ]:
# duplicated participants removed

In [47]:
%%R -i data_no_b
library(nlme)
model <- lme(chose_masc ~ time*prime_, random=~1|participant_id, data=data_no_b)
summary(model)

Linear mixed-effects model fit by REML
  Data: data_no_b 
       AIC      BIC    logLik
  15515.08 15602.89 -7745.539

Random effects:
 Formula: ~1 | participant_id
        (Intercept)  Residual
StdDev:    0.144375 0.4744112

Fixed effects:  chose_masc ~ time * prime_ 
                            Value  Std.Error    DF   t-value p-value
(Intercept)             0.5584862 0.02356291 10845 23.701920  0.0000
time                    0.0626963 0.01990474 10845  3.149815  0.0016
prime_male group       -0.0446732 0.03380979   286 -1.321310  0.1875
prime_male/female      -0.0036359 0.03318966   286 -0.109550  0.9128
prime_male/male        -0.0661722 0.03351696   286 -1.974289  0.0493
prime_pathogen         -0.0125531 0.03348413   286 -0.374896  0.7080
time:prime_male group   0.0120937 0.02860643 10845  0.422763  0.6725
time:prime_male/female -0.0596645 0.02807279 10845 -2.125348  0.0336
time:prime_male/male    0.0020353 0.02836679 10845  0.071749  0.9428
time:prime_pathogen    -0.0252525 0.0283

In [63]:
# first 5 trials removed

In [29]:
%%R -i no_5_trial
library(nlme)
model <- lme(chose_masc ~ time*prime_, random=~1|participant_id, data=no_5_trial)
summary(model)

Linear mixed-effects model fit by REML
  Data: no_5_trial 
       AIC      BIC    logLik
  15583.07 15670.96 -7779.534

Random effects:
 Formula: ~1 | participant_id
        (Intercept)  Residual
StdDev:   0.1456889 0.4729548

Fixed effects:  chose_masc ~ time * prime_ 
                            Value  Std.Error    DF   t-value p-value
(Intercept)             0.5846507 0.02322365 10880 25.174795  0.0000
time                    0.0376025 0.01987491 10880  1.891959  0.0585
prime_male group       -0.0630667 0.03337572   326 -1.889598  0.0597
prime_male/female      -0.0182705 0.03285253   326 -0.556138  0.5785
prime_male/male        -0.0892647 0.03301968   326 -2.703378  0.0072
prime_pathogen         -0.0248005 0.03337990   326 -0.742976  0.4580
time:prime_male group   0.0364158 0.02858826 10880  1.273804  0.2028
time:prime_male/female -0.0583148 0.02814432 10880 -2.071993  0.0383
time:prime_male/male    0.0158903 0.02830620 10880  0.561372  0.5746
time:prime_pathogen    -0.0084455 0.028

### Using lmer

In [8]:
# original data
Lmer("chose_masc ~ time*prime_ + (1|participant_id)", data=data, family = 'binomial').fit()[["Estimate", "SE", "Z-stat", "P-val"]]

Linear mixed model fit by maximum likelihood  ['lmerMod']
Formula: chose_masc~time*prime_+(1|participant_id)

Family: binomial	 Inference: parametric

Number of observations: 12664	 Groups: {'participant_id': 331.0}

Log-likelihood: -8385.239 	 AIC: 16792.479

Random effects:

                       Name    Var    Std
participant_id  (Intercept)  0.375  0.612

No random effect correlations specified

Fixed effects:



,Estimate,SE,Z-stat,P-val
(Intercept),0.249,0.095,2.627,0.009
time,0.291,0.083,3.512,0.000
prime_male group,-0.199,0.136,-1.466,0.143
prime_male/female,-0.047,0.134,-0.352,0.725
prime_male/male,-0.332,0.134,-2.468,0.014
prime_pathogen,-0.054,0.136,-0.394,0.694
time:prime_male group,0.079,0.119,0.664,0.507
time:prime_male/female,-0.291,0.116,-2.495,0.013
time:prime_male/male,0.010,0.117,0.086,0.932
time:prime_pathogen,-0.096,0.119,-0.806,0.420


In [9]:
# duplicated participants removed
Lmer("chose_masc ~ time*prime_ + (1|participant_id)", data=data_no_b, family = 'binomial').fit()[["Estimate", "SE", "Z-stat", "P-val"]]

Linear mixed model fit by maximum likelihood  ['lmerMod']
Formula: chose_masc~time*prime_+(1|participant_id)

Family: binomial	 Inference: parametric

Number of observations: 11141	 Groups: {'participant_id': 291.0}

Log-likelihood: -7355.060 	 AIC: 14732.120

Random effects:

                       Name    Var    Std
participant_id  (Intercept)  0.399  0.632

No random effect correlations specified

Fixed effects:



,Estimate,SE,Z-stat,P-val
(Intercept),0.254,0.104,2.456,0.014
time,0.281,0.089,3.163,0.002
prime_male group,-0.199,0.149,-1.341,0.180
prime_male/female,-0.012,0.146,-0.084,0.933
prime_male/male,-0.283,0.147,-1.924,0.054
prime_pathogen,-0.052,0.147,-0.352,0.725
time:prime_male group,0.054,0.128,0.426,0.670
time:prime_male/female,-0.268,0.125,-2.146,0.032
time:prime_male/male,0.003,0.126,0.021,0.984
time:prime_pathogen,-0.115,0.126,-0.915,0.360


In [10]:
# first 5 trials removed
Lmer("chose_masc ~ time*prime_ + (1|participant_id)", data=no_5_trial, family = 'binomial').fit()[["Estimate", "SE", "Z-stat", "P-val"]]

Linear mixed model fit by maximum likelihood  ['lmerMod']
Formula: chose_masc~time*prime_+(1|participant_id)

Family: binomial	 Inference: parametric

Number of observations: 11216	 Groups: {'participant_id': 331.0}

Log-likelihood: -7387.008 	 AIC: 14796.016

Random effects:

                       Name    Var    Std
participant_id  (Intercept)  0.412  0.642

No random effect correlations specified

Fixed effects:



,Estimate,SE,Z-stat,P-val
(Intercept),0.373,0.103,3.615,0.000
time,0.171,0.089,1.911,0.056
prime_male group,-0.284,0.148,-1.921,0.055
prime_male/female,-0.078,0.146,-0.537,0.591
prime_male/male,-0.389,0.146,-2.660,0.008
prime_pathogen,-0.108,0.148,-0.730,0.465
time:prime_male group,0.162,0.128,1.263,0.207
time:prime_male/female,-0.262,0.126,-2.086,0.037
time:prime_male/male,0.064,0.126,0.504,0.614
time:prime_pathogen,-0.041,0.128,-0.317,0.751


## Fitting other models defined in the original code

In [21]:
# not including any interaction terms
Lmer("chose_masc ~ time + prime_ + (1|participant_id)", data=data, family = 'binomial').fit()[["Estimate", "SE", "Z-stat", "P-val"]]

Linear mixed model fit by maximum likelihood  ['lmerMod']
Formula: chose_masc~time+prime_+(1|participant_id)

Family: binomial	 Inference: parametric

Number of observations: 12664	 Groups: {'participant_id': 331.0}

Log-likelihood: -8391.216 	 AIC: 16796.433

Random effects:

                       Name    Var    Std
participant_id  (Intercept)  0.374  0.612

No random effect correlations specified

Fixed effects:



,Estimate,SE,Z-stat,P-val
(Intercept),0.279,0.087,3.207,0.001
time,0.229,0.037,6.131,0.000
prime_male group,-0.159,0.122,-1.305,0.192
prime_male/female,-0.193,0.120,-1.610,0.107
prime_male/male,-0.326,0.121,-2.702,0.007
prime_pathogen,-0.101,0.122,-0.830,0.406


In [17]:
# including trial in the model
Lmer("chose_masc ~ trial + time*prime_ + (1|participant_id)", data=data, family = 'binomial').fit()[["Estimate", "SE", "Z-stat", "P-val"]]

[1] "Model failed to converge with max|grad| = 0.0199635 (tol = 0.002, component 1)"
[2] " \n"                                                                           

[1] "Model is nearly unidentifiable: very large eigenvalue\n - Rescale variables?"
[2] " \n"                                                                         

Linear mixed model fit by maximum likelihood  ['lmerMod']
Formula: chose_masc~trial_+time*prime_+(1|participant_id)

Family: binomial	 Inference: parametric

Number of observations: 12664	 Groups: {'participant_id': 331.0}

Log-likelihood: -8373.493 	 AIC: 16770.985

Random effects:

                       Name    Var    Std
participant_id  (Intercept)  0.377  0.614

No random effect correlations specified

Fixed effects:



,Estimate,SE,Z-stat,P-val
(Intercept),0.078,0.101,0.776,0.438
trial_,0.016,0.003,4.849,0.000
time,-0.022,0.105,-0.211,0.833
prime_male group,-0.200,0.136,-1.467,0.142
prime_male/female,-0.047,0.134,-0.348,0.728
prime_male/male,-0.331,0.135,-2.463,0.014
prime_pathogen,-0.053,0.136,-0.387,0.699
time:prime_male group,0.079,0.119,0.661,0.508
time:prime_male/female,-0.293,0.117,-2.513,0.012
time:prime_male/male,0.008,0.117,0.072,0.943


In [21]:
# including image as a random factor
Lmer("chose_masc ~ time*prime_ + (1|participant_id) + (1|image)", data=data, family = 'binomial').fit()[["Estimate", "SE", "Z-stat", "P-val"]]

Linear mixed model fit by maximum likelihood  ['lmerMod']
Formula: chose_masc~time*prime_+(1|participant_id)+(1|image)

Family: binomial	 Inference: parametric

Number of observations: 12664	 Groups: {'participant_id': 331.0, 'image': 20.0}

Log-likelihood: -7850.604 	 AIC: 15725.209

Random effects:

                       Name    Var    Std
participant_id  (Intercept)  0.468  0.684
image           (Intercept)  0.457  0.676

No random effect correlations specified

Fixed effects:



,Estimate,SE,Z-stat,P-val
(Intercept),0.270,0.183,1.477,0.140
time,0.324,0.087,3.737,0.000
prime_male group,-0.210,0.148,-1.417,0.157
prime_male/female,-0.049,0.146,-0.333,0.739
prime_male/male,-0.365,0.147,-2.487,0.013
prime_pathogen,-0.052,0.148,-0.354,0.723
time:prime_male group,0.080,0.125,0.640,0.522
time:prime_male/female,-0.322,0.122,-2.638,0.008
time:prime_male/male,0.009,0.123,0.074,0.941
time:prime_pathogen,-0.109,0.124,-0.877,0.380


In [25]:
# including trial in the model and image as a random factor
Lmer("chose_masc ~ trial + time*prime_ + (1|participant_id) + (1|image)", data=data, family = 'binomial').fit()[["Estimate", "SE", "Z-stat", "P-val"]]

Linear mixed model fit by maximum likelihood  ['lmerMod']
Formula: chose_masc~trial+time*prime_+(1|participant_id)+(1|image)

Family: binomial	 Inference: parametric

Number of observations: 12664	 Groups: {'participant_id': 331.0, 'image': 20.0}

Log-likelihood: -7839.608 	 AIC: 15705.217

Random effects:

                       Name    Var    Std
participant_id  (Intercept)  0.470  0.686
image           (Intercept)  0.457  0.676

No random effect correlations specified

Fixed effects:



,Estimate,SE,Z-stat,P-val
(Intercept),0.097,0.187,0.518,0.605
trial,0.016,0.003,4.698,0.000
time,0.006,0.110,0.053,0.958
prime_male group,-0.210,0.149,-1.416,0.157
prime_male/female,-0.047,0.146,-0.321,0.748
prime_male/male,-0.364,0.147,-2.475,0.013
prime_pathogen,-0.051,0.149,-0.343,0.732
time:prime_male group,0.080,0.125,0.643,0.520
time:prime_male/female,-0.325,0.122,-2.658,0.008
time:prime_male/male,0.006,0.123,0.052,0.959


In [22]:
# including image as a random nested factor with participants
Lmer("chose_masc ~ trial + time*prime_ + (1|participant_id:image)", data=data, family = 'binomial').fit()[["Estimate", "SE", "Z-stat", "P-val"]]

[1] "Model failed to converge with max|grad| = 0.044463 (tol = 0.002, component 1)"
[2] " \n"                                                                          

[1] "Model is nearly unidentifiable: very large eigenvalue\n - Rescale variables?"
[2] " \n"                                                                         

Linear mixed model fit by maximum likelihood  ['lmerMod']
Formula: chose_masc~trial+time*prime_+(1|participant_id:image)

Family: binomial	 Inference: parametric

Number of observations: 12664	 Groups: {'participant_id:image': 6604.0}

Log-likelihood: -8448.114 	 AIC: 16920.229

Random effects:

                             Name    Var    Std
participant_id:image  (Intercept)  1.526  1.236

No random effect correlations specified

Fixed effects:



,Estimate,SE,Z-stat,P-val
(Intercept),0.110,0.085,1.291,0.197
trial,0.019,0.004,4.604,0.000
time,-0.005,0.121,-0.039,0.969
prime_male group,-0.248,0.106,-2.346,0.019
prime_male/female,-0.053,0.104,-0.515,0.607
prime_male/male,-0.416,0.105,-3.977,0.000
prime_pathogen,-0.068,0.106,-0.640,0.522
time:prime_male group,0.101,0.132,0.763,0.446
time:prime_male/female,-0.360,0.130,-2.774,0.006
time:prime_male/male,0.014,0.130,0.110,0.912


## Fitting a random coefficient model

The original study fit only a random intercept model. Here I also fit a random slope, reflecting how participants' masculinity preferences change individually over time.

In [11]:
# original data
Lmer("chose_masc ~ time*prime_ + (1 + time|participant_id)", data=data, family = 'binomial').fit()[["Estimate", "SE", "Z-stat", "P-val"]]

Linear mixed model fit by maximum likelihood  ['lmerMod']
Formula: chose_masc~time*prime_+(1+time|participant_id)

Family: binomial	 Inference: parametric

Number of observations: 12664	 Groups: {'participant_id': 331.0}

Log-likelihood: -8378.674 	 AIC: 16783.348

Random effects:

                       Name    Var   Std
participant_id  (Intercept)  0.281  0.53
participant_id         time  0.029  0.17

                        IV1   IV2  Corr
participant_id  (Intercept)  time   1.0

Fixed effects:



,Estimate,SE,Z-stat,P-val
(Intercept),0.244,0.087,2.819,0.005
time,0.308,0.086,3.598,0.000
prime_male group,-0.194,0.124,-1.558,0.119
prime_male/female,-0.047,0.122,-0.381,0.703
prime_male/male,-0.326,0.123,-2.656,0.008
prime_pathogen,-0.053,0.124,-0.424,0.672
time:prime_male group,0.070,0.123,0.567,0.571
time:prime_male/female,-0.299,0.120,-2.486,0.013
time:prime_male/male,-0.002,0.121,-0.015,0.988
time:prime_pathogen,-0.100,0.122,-0.821,0.412


## Using other priming conditions as the 'reference group'

In [13]:
data_ref = data.copy()
data_ref["prime_"] = data_ref["prime_"].replace({"_neutral" : "neutral"})

In [17]:
# male/male as reference
data_mm = data_ref.copy()
data_mm["prime_"] = data_mm["prime_"].replace({"male/male" : "_male/male"})
Lmer("chose_masc ~ time*prime_ + (1|participant_id)", data=data_mm, family = 'binomial').fit()[["Estimate", "SE", "Z-stat", "P-val"]]

Linear mixed model fit by maximum likelihood  ['lmerMod']
Formula: chose_masc~time*prime_+(1|participant_id)

Family: binomial	 Inference: parametric

Number of observations: 12664	 Groups: {'participant_id': 331.0}

Log-likelihood: -8385.239 	 AIC: 16792.479

Random effects:

                       Name    Var    Std
participant_id  (Intercept)  0.375  0.612

No random effect correlations specified

Fixed effects:



,Estimate,SE,Z-stat,P-val
(Intercept),-0.083,0.095,-0.870,0.384
time,0.301,0.083,3.634,0.000
prime_male group,0.132,0.136,0.969,0.333
prime_male/female,0.284,0.134,2.119,0.034
prime_neutral,0.332,0.134,2.468,0.014
prime_pathogen,0.278,0.136,2.039,0.041
time:prime_male group,0.069,0.119,0.580,0.562
time:prime_male/female,-0.301,0.116,-2.581,0.010
time:prime_neutral,-0.010,0.117,-0.086,0.932
time:prime_pathogen,-0.106,0.119,-0.891,0.373


In [18]:
# male group as reference
data_mg = data_ref.copy()
data_mg["prime_"] = data_mg["prime_"].replace({"male group" : "_male group"})
Lmer("chose_masc ~ time*prime_ + (1|participant_id)", data=data_mg, family = 'binomial').fit()[["Estimate", "SE", "Z-stat", "P-val"]]

Linear mixed model fit by maximum likelihood  ['lmerMod']
Formula: chose_masc~time*prime_+(1|participant_id)

Family: binomial	 Inference: parametric

Number of observations: 12664	 Groups: {'participant_id': 331.0}

Log-likelihood: -8385.239 	 AIC: 16792.479

Random effects:

                       Name    Var    Std
participant_id  (Intercept)  0.375  0.612

No random effect correlations specified

Fixed effects:



,Estimate,SE,Z-stat,P-val
(Intercept),0.049,0.098,0.504,0.614
time,0.370,0.086,4.322,0.000
prime_male/female,0.152,0.136,1.121,0.262
prime_male/male,-0.132,0.136,-0.969,0.333
prime_neutral,0.199,0.136,1.466,0.143
prime_pathogen,0.146,0.138,1.057,0.290
time:prime_male/female,-0.370,0.118,-3.121,0.002
time:prime_male/male,-0.069,0.119,-0.580,0.562
time:prime_neutral,-0.079,0.119,-0.664,0.507
time:prime_pathogen,-0.175,0.121,-1.450,0.147


In [19]:
# male/female as reference
data_mf = data_ref.copy()
data_mf["prime_"] = data_mf["prime_"].replace({"male/female" : "_male/female"})
Lmer("chose_masc ~ time*prime_ + (1|participant_id)", data=data_mf, family = 'binomial').fit()[["Estimate", "SE", "Z-stat", "P-val"]]

Linear mixed model fit by maximum likelihood  ['lmerMod']
Formula: chose_masc~time*prime_+(1|participant_id)

Family: binomial	 Inference: parametric

Number of observations: 12664	 Groups: {'participant_id': 331.0}

Log-likelihood: -8385.239 	 AIC: 16792.479

Random effects:

                       Name    Var    Std
participant_id  (Intercept)  0.375  0.612

No random effect correlations specified

Fixed effects:



,Estimate,SE,Z-stat,P-val
(Intercept),0.202,0.095,2.132,0.033
time,0.000,0.082,0.002,0.998
prime_male group,-0.152,0.136,-1.121,0.262
prime_male/male,-0.284,0.134,-2.119,0.034
prime_neutral,0.047,0.134,0.352,0.725
prime_pathogen,-0.006,0.136,-0.048,0.962
time:prime_male group,0.370,0.118,3.120,0.002
time:prime_male/male,0.301,0.116,2.581,0.010
time:prime_neutral,0.291,0.116,2.495,0.013
time:prime_pathogen,0.195,0.118,1.653,0.098


In [25]:
# pathogen as reference
data_p = data_ref.copy()
data_p["prime_"] = data_p["prime_"].replace({"pathogen" : "_pathogen"})
Lmer("chose_masc ~ time*prime_ + (1|participant_id)", data=data_p, family = 'binomial').fit()[["Estimate", "SE", "Z-stat", "P-val"]]

Linear mixed model fit by maximum likelihood  ['lmerMod']
Formula: chose_masc~time*prime_+(1|participant_id)

Family: binomial	 Inference: parametric

Number of observations: 12664	 Groups: {'participant_id': 331.0}

Log-likelihood: -8385.239 	 AIC: 16792.479

Random effects:

                       Name    Var    Std
participant_id  (Intercept)  0.375  0.612

No random effect correlations specified

Fixed effects:



,Estimate,SE,Z-stat,P-val
(Intercept),0.195,0.097,2.002,0.045
time,0.195,0.085,2.299,0.022
prime_male group,-0.146,0.138,-1.057,0.290
prime_male/female,0.006,0.136,0.048,0.962
prime_male/male,-0.278,0.136,-2.039,0.041
prime_neutral,0.054,0.136,0.394,0.693
time:prime_male group,0.175,0.121,1.449,0.147
time:prime_male/female,-0.195,0.118,-1.653,0.098
time:prime_male/male,0.106,0.119,0.892,0.373
time:prime_neutral,0.096,0.119,0.806,0.420


## Discussing the results

Summarising the results from the different models: only variables which had a significant p-value in at least one model are included: all other variables can be assumed to have never been significant.

In [32]:
model_results = pd.read_csv("modelling_results.csv", header=[0, 1])
model_results

time          \
                                         model        AIC   coef p-value   
0                            lme original data  17676.660  0.065  0.0050   
1                       lme duplicates removed  15515.080  0.063  0.0016   
2                   lme first 5 trials removed  15583.070  0.038  0.0580   
3                           lmer original data  16792.479  0.291  0.0000   
4                      lmer duplicates removed  14732.120  0.281  0.0020   
5                  lmer first 5 trials removed  14796.016  0.171  0.0560   
6                         no interaction terms  16796.433  0.229  0.0000   
7                     random coefficient model  16783.348  0.308  0.0000   
8                              image as random  15725.209  0.324  0.0000   
9                              including trial  16770.985 -0.022  0.8330   
10         including trial and image as random  15705.217  0.006  0.9580   
11  including trial and image as nested random  16920.229 -0.005  0.9690   

    trial         male/male main effect         time:male/female interaction  \
     coef p-value                  coef p-value                         coef   
0     NaN     NaN                -0.077  0.0123                       -0.065   
1     NaN     NaN                -0.066  0.0493                       -0.060   
2     NaN     NaN                -0.089  0.0070                       -0.058   
3     NaN     NaN                -0.332  0.0140                       -0.291   
4     NaN     NaN                -0.283  0.0540                       -0.268   
5     NaN     NaN                -0.389  0.0080                       -0.262   
6     NaN     NaN                -0.326  0.0070                          NaN   
7     NaN     NaN                -0.326  0.0080                       -0.299   
8     NaN     NaN                -0.365  0.0130                       -0.322   
9   0.016     0.0                -0.331  0.0140                       -0.293   
10  0.016     0.0                -0.364  0.0130                       -0.325   
11  0.019     0.0                -0.416  0.0000                       -0.360   

            
   p-value  
0   0.0136  
1   0.0336  
2   0.0383  
3   0.0130  
4   0.0320  
5   0.0370  
6      NaN  
7   0.0130  
8   0.0080  
9   0.0120  
10  0.0080  
11  0.0060

Summarising the results from changing the reference variable when fitting the original model. A few variables only have been selected:

In [19]:
ref_change = pd.read_csv("changing_reference.csv", header=[0, 1])
ref_change

intercept           time         neutral main effect          \
     reference      coef p-value   coef p-value                coef p-value   
0      neutral     0.249   0.009  0.291   0.000                 NaN     NaN   
1    male/male    -0.083   0.384  0.301   0.000               0.332   0.014   
2   male group     0.049   0.614  0.370   0.000               0.199   0.143   
3  male/female     0.202   0.033  0.000   0.998               0.047   0.725   
4     pathogen     0.195   0.045  0.195   0.022               0.054   0.693   

  male/male main effect         pathogen main effect          \
                   coef p-value                 coef p-value   
0                -0.332   0.014              -0.0400   0.694   
1                   NaN     NaN               0.3278   0.041   
2                -0.132   0.333               0.1460   0.290   
3                -0.284   0.034              -0.0060   0.962   
4                -0.278   0.041                  NaN     NaN   

  male/female main effect         time:male/female interaction          
                     coef p-value                         coef p-value  
0                  -0.047   0.725                       -0.291   0.013  
1                   0.284   0.034                       -0.301   0.010  
2                   0.152   0.262                       -0.370   0.002  
3                     NaN     NaN                          NaN     NaN  
4                   0.006   0.962                       -0.195   0.098

### Choice of model

Many different forms of models were tested, however the results from each were largely the same: the main effect of "time" (pre-post prime) or "trial" was significant, as was the main effect for "male/male" priming group, and the interaction between time:male/female priming group.

Even when the duplicated participants were removed from the dataset and the model fitted again, the same set of variables remained significant. However the strength of the significance is reduced:  
In the original data, the interaction of time with male/female priming group has a p-value of 0.01 .  
When the duplicated participants are removed, the interaction of time with male/female priming group has a p-value of 0.03 .  

There is certainly a statistically significant effect found in the dataset. Whether this effect is generalisable and corresponds to a real world effect, is more dubious.

### Interpreting main effects

**The main effect of the priming groups** reflect the effect of a participant being in that priming group compared to the reference priming group (neutral), while the other categorical variables are set to 0 (pairwise comparisons). This means that "time" (pre-post priming time effect) is also set to 0, i.e. we only see the effect at the pre-prime state.

Therefore the main effect coefficient gives us no information about the effect of the priming stimulus. The fact that the p-value for the male/male main effect is significant, reflects only that the participants randomly assigned to the male/male priming group happened to naturally have a lower preference for masculinity than those assigned to the reference group. This finding was also seen and shown to be statistically signficant in the Notebook "2. Masculinity preferences by priming group."

**The main effect of time (pre/post prime)** indicates the effect of being pre or post prime on mean masculinity preferences, for participants in the reference group.

We therefore see a significant main effect of time (or trial) in every model, expect for that which uses the male/female priming group as the reference. This effect was also demonstrated in the Notebook "3. Masculinity preferences by time", where a linear regression for trial number predicting masculinity preference, only on participants in the male/female priming group, showed no significant slope.

### Interpreting the interactions

Because we are interested in the way that being in a particular priming group modifies the way in which time (pre/post prime) affects masculinity preferences, only the interaction terms between time\*priming group are relevant for the outcomes of the study.

**The time:male/female interaction** describes the difference in the effect of time on mean masculinity preferences, between those in the male/female priming group, and those in the reference group. No matter which reference group was used, this difference was almost always significant, (and no other interactions were significant), and the respective coefficient always negative. The negative coefficient suggests that post-prime (thus time=1) the mean masculinity preference for those in the male/female group is lower than for those in the reference group.

The only exception was when the "pathogen" priming group was used as a reference; then the time:male/female interaction was not significant. This can be explained via the plot in Notebook "3. Masculinity preferences by time", where we can see that the slope describing masculinity preference over time for only the pathogen priming group was not particularly steep, much like the slope for the male/female priming group. The difference between these two slopes were not statistically significant.

When the male/female priming group is used as a reference, all of the other time:priming group interactions (except pathogen) are significant, reflecting in the different way that the other priming groups interact with time, compared to the male/female group.